<a href="https://colab.research.google.com/github/samuelyota/TA/blob/main/Deteksi_Kepadatan_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python --version


Python 3.10.11


In [ ]:
import cv2
import numpy as np
import time

In [ ]:
cap = cv2.VideoCapture("video1.mp4")

In [ ]:
path_label = 'coco.names'

In [ ]:
classes = []
with open (path_label, 'rt') as f:
  classes = f.read().rstrip('\n').split('\n')

In [ ]:
wght_hght_target = 416
model_Config = 'yolov3.cfg'
model_Weights = 'yolov3.weights'
confThreshold = 0.4
nmsThreshold = 0.2
inccount1 = 0
inccount2 = 0
inccount3 = 0
inccount4 = 0
inccount5 = 0
inccount_reset = 0
start_time = time.time()

In [ ]:
network = cv2.dnn.readNetFromDarknet(model_Config, model_Weights)
network.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
network.setPreferableBackend(cv2.dnn.DNN_TARGET_CPU)

In [ ]:
result = cv2.VideoWriter('result.mp4',cv2.VideoWriter_fourcc(*'XVID'),20,(640,480))

In [ ]:
def findObject(outputs,img):
  heightTar, weightTar, chanelsTar = img.shape
  bbox = []
  classId = []
  confidence = []
  count1 = 0
  count2 = 0
  count3 = 0
  count4 = 0
  for output in outputs:
    for det in output :
      score = det [5:]
      classIds = np.argmax(score)
      confids = score [classIds]
      if classIds == 2 or classIds == 3 or classIds == 7 or classIds == 5:
        if confids > confThreshold:
          x,h = int(det[2]*weightTar), int(det[2]*heightTar)
          x,y = int((det[0]*weightTar)-w/2), int((det[3]*heightTar)-h/2)
          bbox.append([x,y,w,h])
          classId.append(classIds)
          confidance.append(float(confids))
          if (int(img.shape[0]/2)-3) < y < (int(img.shape[0]/2)+3):
            if classes == 2 :
             count1 = count1 +1
            if classes == 3 :
              count2 = count2 +1
            if classes == 5 :
              count3 = count3 +1
            if classes == 7 :
              count4 = count4 +1
      else:
        continue
  draw_box = cv2.dnn.NMSBoxes[bbox, confidance, confThreshold, nmsThreshold]
  for i in draw_box :
    i = i[0]
    box = bbox[1]
    x,y,w,h = box[0], box[1],box[2],box[3]
    cv2.rectangle(img,(x,y),(x+w, y+h),(255,255,0),2)
    cv2.putText(img,f'{classes[classId[i]].upper()} {int(confidance(i)*100)}%',(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,255,0),2)

    cv2.line(img, (0,int(img.shape[0]/2)+3), (int(img.shape[1]),int(img.shape[0]/2)+3), (0,0,100),1)
    cv2.line(img, (0,int(img.shape[0]/2) - 3), (int(img.shape[1]),int(img.shape[0] / 2)-3), (0,0,100),1)

    return count1, count2, count3, count4

In [ ]:
while True:
  success, img = cap.read()
  img = cv2.resize(img, (640,480))
  cv2.imshow('video',img)
  blob = cv2.dnn.blobFromImage(img,1/255, (wght_hght_target,wght_hght_target),[0,0,0],1,crop=False)
  network.setInput(blob)
  LayerNames = network.getLayerNames()
  outputName = [LayerNames[i[0]-1] for i in network.getUnconnectedOutLayers()]
  outputs = network.forward(outputName)
  couter1,counter2,counter3,counter4 = findObject(outputs, img)

  inccount1 = inccount1 + counter1
  inccount2 = inccount2 + counter2
  inccount3 = inccount3 + counter3
  inccount4 = inccount4 + counter4
  inccount5 = inccount5 + (counter1+counter2+counter3+counter4)
  real_time = time.time()
  inccount_reset = int(time.time()-start_time)
  if inccount_reset == 3600 :
    inccount1 = 0
    inccount2 = 0
    inccount3 = 0
    inccount4 = 0
    inccount5 = 0
    inccount_reset = 0
    start_time = run_time

  cv2.putText(img,f'counting car : (inccount1)',(25,20),cv2.FONT_HERSHEY_SIMPLEX,0.6,(200,0,0),2) 
  cv2.putText(img,f'counting Bike : (inccount2)',(25,40),cv2.FONT_HERSHEY_SIMPLEX,0.6,(0,200,0),2)
  cv2.putText(img,f'counting Bus : (inccount3)',(25,60),cv2.FONT_HERSHEY_SIMPLEX,0.6,(200,0,0),2)
  cv2.putText(img,f'counting Truck : (inccount4)',(25,80),cv2.FONT_HERSHEY_SIMPLEX,0.6,(0,200,200),2)
  cv2.putText(img,f'Total : (inccount5)',(25,100),cv2.FONT_HERSHEY_SIMPLEX,0.6,(200,200,0),2)
  result.write(img)
  cv2.imshow('video',img)

  if cv2.waitkey(1) & 0xFF == ord('q'):
    break

DisabledFunctionError: ignored

In [ ]:
cap.release()
result.release()
cv2.destroyAllWindows()